In [ ]:
import torch
from torchvision import transforms
from transformers import SamModel, SamProcessor
from diffusers.utils import load_image, make_image_grid
import matplotlib.pyplot as plt
from PIL import Image

def get_shoulder_coordinates(image_path):
    # Initialize MediaPipe Pose
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose()

    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Could not load image.")
        return None

    # Convert the image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the image
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        height, width, _ = image.shape
        landmarks = results.pose_landmarks.landmark

        # Extract left and right shoulder coordinates (landmarks 11 and 12)
        left_shoulder = (int(landmarks[11].x * width), int(landmarks[11].y * height))
        right_shoulder = (int(landmarks[12].x * width), int(landmarks[12].y * height))

        return left_shoulder, right_shoulder
    else:
        print("No pose detected in the image.")
        return None

image_path = "/content/drive/MyDrive/Try On/Harsh.jpg"  # Change this to your image file path
coordinates = get_shoulder_coordinates(image_path)

# Load original image
img = load_image(image_path)

# Load new t-shirt image (ensure it has a transparent background)
new_tshirt = load_image("/content/drive/MyDrive/Try On/Garment-1.png")

# Load model and processor
model = SamModel.from_pretrained("Zigeng/SlimSAM-uniform-50").to("cuda")
processor = SamProcessor.from_pretrained("Zigeng/SlimSAM-uniform-50")

# Define input points for segmentation (adjust as needed)
left_shoulder, right_shoulder = coordinates
input_points = [[[left_shoulder[0], left_shoulder[1]], [right_shoulder[0], right_shoulder[1]]]]

inputs = processor(img, input_points=input_points, return_tensors="pt").to("cuda")
outputs = model(**inputs)

# Process mask
masks = processor.image_processor.post_process_masks(outputs.pred_masks.cpu(),
                                                      inputs["original_sizes"].cpu(),
                                                      inputs["reshaped_input_sizes"].cpu())

# Convert mask to PIL image
mask_tensor = masks[0][0][2].to(dtype=torch.uint8)
mask = transforms.ToPILImage()(mask_tensor * 255)

# Resize new t-shirt to match the original image
new_tshirt = new_tshirt.resize(img.size, Image.LANCZOS)

# Paste new t-shirt onto the original image using the mask
img_with_new_tshirt = Image.composite(new_tshirt, img, mask)

# Display results
make_image_grid([img, mask, img_with_new_tshirt], cols=3, rows=1)
